- #### Họ và tên: Võ Nhất Huy 
- #### MSSV: 19127642

***
### CÁCH CHẠY FILE
- Markdown cell 3 cells thu thập dữ liệu tuần (3 cells cuối) khi thu thập dữ liệu cho 2 ngày, markdown 3 cells phía trên khi muốn thu thập dữ liệu tuần
- Restart & run all  
***

# THU THẬP DỮ LIỆU
***


In [7]:
import time
import datetime
import csv
import pandas as pd 
import re
import os
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By

In [8]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Oaki\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [9]:
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))

In [10]:
def parse_country_data(country_text):
  lines = country_text.split("\n")
  data = []
  for each_line in lines:
    try:
      result = re.search(">(.*?)</", each_line)
      if ">" in result.group(1):
        result = re.search(">(.*?)$", result.group(1))

      data.append(result.group(1))
    except:
      continue
  return data

In [11]:
def crawl_raw_data(soup):
    df = pd.DataFrame(columns = ["#", "Date", "Country, Other", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Serious, Critical", "Tot Cases/1M Pop", "Deaths/1M Pop", "Total Tests", "Tests/1M Pop", "Population", "Continent"])
    country_odd_data = soup.find_all("tr", attrs={"class":"odd"})
    country_even_data = soup.find_all("tr", attrs={"class":"even"})
    for i in range(len(country_even_data)):
        date = datetime.datetime.today() - datetime.timedelta(days=(len(df.index)+1)//224)
        result = parse_country_data(str(country_odd_data[i]))
        date = date.strftime('%d:%m:%Y')
        if result[0] != '':
            result.insert(1, date)
            df_length = len(df)
            df.loc[df_length] = result  
        result = parse_country_data(str(country_even_data[i]))
        if result[0] != '':
            result.insert(1, date)
            df_length = len(df)
            df.loc[df_length] = result  
    #for i in range(len(country_even_data)):
    return [df.iloc[448:], df.iloc[224:448]]

In [13]:
driver.get("https://www.worldometers.info/coronavirus/")

html = driver.page_source


In [14]:
soup = BeautifulSoup(html, 'html.parser')
df = crawl_raw_data(soup)

In [15]:
if not os.path.isfile('Corona_day.tsv'):
    df[0].to_csv("Corona_day.tsv", sep = "\t", mode = "a", index = False)
else:
    df[0].to_csv("Corona_day.tsv", sep = "\t", mode = "a", index = False, header = False)
df[1].to_csv("Corona_day.tsv", sep = "\t", mode = "a", index = False, header = False)
